# Asset Calendars

In [1]:
import numpy as np
from numba import njit, types
from numba.typed import Dict
import time
from pathlib import Path
from typing import Any
import yaml

# Import loader for real data experiments
from specparser.amt import loader, table



In [2]:
# Get asset data for a specific asset
_DAYS_PER_MONTH = (31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
amt_path = "../data/amt.yml"
amt_resolved = Path(amt_path).resolve()
with open(amt_resolved, "r") as f: run_options = yaml.safe_load(f)
amt = run_options.get("amt", {})
expiry_schedules = run_options.get("expiry_schedules")
underlying_map: dict[str, dict[str, Any]] = {}
for asset_data in amt.values():
    if isinstance(asset_data, dict):
        underlying = asset_data.get("Underlying")
        if underlying and asset_data.get("WeightCap")>0:
            underlying_map[underlying] = asset_data

In [6]:
start_time = time.perf_counter()

year_start = 2001
month_start = 1
year_month_start = year_start * 12 + month_start - 1
year_end = 2026
month_end = 1
year_month_end = year_end * 12 + month_end - 1
month_count = year_month_end - year_month_start + 1

assets = list(underlying_map.keys())
#assets = ["EURUSD Curncy","LA Comdty TKRZ","LA Comdty","IBOXUMAE Curncy", "TSLA US Equity"]

monthly_straddle_count= sum([len(expiry_schedules[underlying_map[asset].get("Options")]) for asset in assets])
straddle_count = monthly_straddle_count * month_count
print(f"total straddles: {straddle_count}")

asset_vec  = np.full(straddle_count,"",dtype=np.dtypes.StringDType())
year_vec   = np.full(straddle_count, np.uint16(0), dtype=np.int16)
month_vec   = np.full(straddle_count, np.uint8(0), dtype=np.int8)
schcnt_vec = np.full(straddle_count, np.uint8(0), dtype=np.int8)
schid_vec  = np.full(straddle_count, np.uint8(0), dtype=np.int8)
ntrc_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
ntrv_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
xprc_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
xprv_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
wgt_vec    = np.full(straddle_count, "", dtype=np.dtypes.StringDType())

hedge1t_vec    = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge2t_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge3t_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge4t_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
volt_vec      = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge1f_vec    = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge2f_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge3f_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
hedge4f_vec   = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
volf_vec      = np.full(straddle_count, "", dtype=np.dtypes.StringDType())
wgt_vec      = np.full(straddle_count, "", dtype=np.dtypes.StringDType())

days0_vec = np.full(straddle_count, np.uint8(0), dtype=np.int8)
days1_vec = np.full(straddle_count, np.uint8(0), dtype=np.int8)
days2_vec = np.full(straddle_count, np.uint8(0), dtype=np.int8)
day_count_vec = np.full(straddle_count, np.uint8(0), dtype=np.int8)

j=-1
for year_month in range( year_start * 12 + month_start - 1, year_end * 12 + month_end - 1 + 1 ):

    year, month =  year_month // 12, year_month % 12 + 1

    for asset in assets:

        assid = np.sum(np.frombuffer(asset.encode('ascii'),dtype=np.uint8))
        asset_data = underlying_map[asset]
        schedule_name = asset_data.get("Options")
        raw_schedule = expiry_schedules.get(schedule_name, []) if schedule_name else []
        schcnt = len(raw_schedule)

        for i, schedule in enumerate(raw_schedule):
            j=j+1
            year_vec[j] = year
            month_vec[j] = month
            asset_vec[j] = asset
            schcnt_vec[j] = schcnt
            schid_vec[j] = i
            schid = i + 1  # 1-based index for fix_value
            parts = schedule.split("_")
            if len(parts) < 3: continue
            # entry_part is just ntrc (a letter like "N"), ntrv follows
            ntrc_vec[j] = parts[0][:1]
            ntrv_vec[j] = parts[0][1:]
            # expiry_part parsing
            expiry_part = parts[1]
            if parts[1] == "OVERRIDE":
                xprc_vec[j] = "OVERRIDE"
                xprv_vec[j] = ""
            elif parts[1][:2] == "BD":
                # BD schedule: xprc="BD", xprv is single letter a/b/c/d after "BD"
                xprc_vec[j] = "BD"
                if parts[1][2] in ["a","b","c","d"]:
                    xprv_vec[j] = str( i * (20 // (schcnt + 1)) + assid % 5 + 1 )
                else:
                    xprv_vec[j] = parts[1][2:]
            elif parts[1][0] in ("F", "R", "W"):
                xprc_vec[j] = parts[1][0]
                xprv_vec[j] = parts[1][1:]
            else:
                xprc_vec[j] = parts[1]
                xprv_vec[j] = ""
            
            wgt_vec[j] = parts[2]

            hedge = asset_data["Hedge"]
            if hedge is None: continue
            vol = asset_data["Vol"]
            if vol is None: continue

            if ntrc_vec[i]=="F" and vol["Source"]!="BBG_LMEVOL":
                if vol.get("Far","")=="": continue
                if vol.get("Near","")==vol.get("Far",""): continue
                if vol.get("Far","")=="NONE": continue

            if vol["Source"]=="BBG" and ntrc_vec[j]=="N":
                volt_vec[j] = vol["Ticker"]
                volf_vec[j] = vol["Near"]

            if vol["Source"]=="BBG" and ntrc_vec[j]=="F":
                volt_vec[j] = vol["Ticker"]
                volf_vec[j] = vol["Far"]

            if vol["Source"]=="CV":
                volt_vec[j] = vol["Near"]
                volf_vec[j] = "none"

            if vol["Source"]=="BBG_LMEVOL":
                fut_month_code = hedge["fut_month_map"][month - 1]
                opt_month_code = "FGHJKMNQUVXZ"[month - 1]
                year_offset = max(1 if fut_month_code < opt_month_code else 0, hedge["min_year_offset"])
                volt_vec[j] = hedge["fut_code"]+"R"+fut_month_code+str(year + year_offset)+" " + hedge["market_code"] 
                volf_vec[j] = "PX_LAST"
            
            if hedge["Source"]=="nonfut":
                hedge1t_vec[j] =  hedge["Ticker"]
                hedge1f_vec[j] = hedge["Field"]

            if hedge["Source"]=="fut":
                fut_month_code = hedge["fut_month_map"][month - 1]
                opt_month_code = "FGHJKMNQUVXZ"[month - 1]
                year_offset = max(1 if fut_month_code < opt_month_code else 0, hedge["min_year_offset"])
                hedge1t_vec[j] = hedge["fut_code"]+fut_month_code+str(year + year_offset)+" " + hedge["market_code"] 
                hedge1f_vec[j] = "PX_LAST"
            

            if hedge["Source"]=="cds":
                hedge1t_vec[j] =  hedge["hedge"]
                hedge1f_vec[j] = "PX_LAST"
                hedge2t_vec[j] =  hedge["hedge1"]
                hedge2f_vec[j] = "PX_LAST"

            if hedge["Source"]=="calc":
                hedge1t_vec[j] = hedge['ccy']+"_fsw0m_"+str(hedge['tenor'])
                hedge2t_vec[j] = hedge['ccy']+"_fsw6m_"+str(hedge['tenor'])
                hedge3t_vec[j] = hedge['ccy']+"_pva0m_"+str(hedge['tenor'])
                hedge4t_vec[j] = hedge['ccy']+"_pva6m_"+str(hedge['tenor'])
                hedge1f_vec[j] = ""
                hedge2f_vec[j] = ""
                hedge3f_vec[j] = ""
                hedge4f_vec[j] = ""

            year0, month0 = (year*12+(month-1)-0) // 12, (year*12+(month-1)-0) % 12 + 1
            year1, month1 = (year*12+(month-1)-1) // 12, (year*12+(month-1)-1) % 12 + 1
            year2, month2 = (year*12+(month-1)-2) // 12, (year*12+(month-1)-2) % 12 + 1

            days0_vec[j] = 29 if (year0 % 4 == 0 and (year0 % 100 != 0 or year0 % 400 == 0)) and (month0==2) else _DAYS_PER_MONTH[month0 - 1]
            days1_vec[j] = 29 if (year1 % 4 == 0 and (year1 % 100 != 0 or year1 % 400 == 0)) and (month1==2) else _DAYS_PER_MONTH[month1 - 1]
            days2_vec[j] = 29 if (year2 % 4 == 0 and (year2 % 100 != 0 or year2 % 400 == 0)) and (month2==2) else _DAYS_PER_MONTH[month2 - 1]

            if ntrc_vec[j]=="N":
                day_count_vec[j] = days0_vec[j] + days1_vec[j]
            elif ntrc_vec[j]=="F":
                day_count_vec[j] = days0_vec[j] + days1_vec[j] + days2_vec[j]
            else:
                day_count_vec[j]=0

   
end_time = time.perf_counter()

print(f"elapsed: {1e3*(end_time-start_time):0.3f}ms")

result = {
    "orientation": "numpy",
    "columns": [
        "year",
        "month",
        "asset", 
        "schcnt", "schid", 
        "ntrc", "ntrv", "xprc", "xprv", "wgt", 
        "hedge1t", "hedge1f", 
        "hedge2t", "hedge2f",
        "hedge3t", "hedge3f",
        "hedge4t", "hedge4f",
        "volt", "volf",
        "days0","days1","days2",
        "day_count",
    ],
    "rows": [
        year_vec,
        month_vec,
        asset_vec,
        schcnt_vec,
        schid_vec ,
        ntrc_vec,
        ntrv_vec,
        xprc_vec,
        xprv_vec,
        wgt_vec,
        hedge1t_vec,
        hedge1f_vec, 
        hedge2t_vec,  
        hedge2f_vec, 
        hedge3t_vec, 
        hedge3f_vec,  
        hedge4t_vec,
        hedge4f_vec,
        volt_vec,
        volf_vec, 
        days0_vec,days1_vec,days2_vec,
        day_count_vec,
    ]

}

table.show_table(result)

total straddles: 223041
elapsed: 818.367ms


,year,month,asset,schcnt,schid,ntrc,ntrv,xprc,xprv,wgt,...,hedge3t,hedge3f,hedge4t,hedge4f,volt,volf,days0,days1,days2,day_count
0,2001,1,LA Comdty,4,0,N,0,OVERRIDE,,33.3,...,,,,,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF,31,31,30,62
1,2001,1,LA Comdty,4,1,N,5,OVERRIDE,,33.3,...,,,,,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF,31,31,30,62
2,2001,1,LA Comdty,4,2,F,10,OVERRIDE,,12.5,...,,,,,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF,31,31,30,92
3,2001,1,LA Comdty,4,3,F,15,OVERRIDE,,12.5,...,,,,,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF,31,31,30,92
4,2001,1,LP Comdty,4,0,N,0,OVERRIDE,,33.3,...,,,,,LP1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF,31,31,30,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223036,2026,1,DVN US Equity,4,3,F,15,F,3,12.5,...,,,,,DVN US Equity,60DAY_IMPVOL_100.0%MNY_DF,31,31,30,92
223037,2026,1,LOW US Equity,4,0,N,0,F,3,33.3,...,,,,,LOW US Equity,30DAY_IMPVOL_100.0%MNY_DF,31,31,30,62
223038,2026,1,LOW US Equity,4,1,N,5,F,3,33.3,...,,,,,LOW US Equity,30DAY_IMPVOL_100.0%MNY_DF,31,31,30,62
223039,2026,1,LOW US Equity,4,2,F,10,F,3,12.5,...,,,,,LOW US Equity,60DAY_IMPVOL_100.0%MNY_DF,31,31,30,92
